In [45]:
import pandas as pd
import numpy as np

In [ ]:
with open('../data/co2_emissions.csv') as f:
    df = pd.read_csv(f)

Group the data by `country_code`, sort each group by year, compute the anomaly using the LLR function, run permutation testing, filter for significant anomalies (p-value < 0.05), and finally sort the results by LLR in descending order.

In [47]:
def compute_llr(x):
    """
    Compute the maximum log-likelihood ratio (LLR) for a change point in a 1D array x.
    Returns the best LLR score, the index of the best change point, and the anomalous subset S*.
    
    Parameters:
        x (np.array): Array of numeric data.
    
    Returns:
        best_llr (float): Maximum improvement in fit (LLR score).
        best_t (int): Index of the best change point.
        best_subset (np.array): The anomalous subset, i.e. x[:best_t].
    """
    n = len(x)
    if n < 2:
        return None, None, None

    mean_all = np.mean(x)
    sse_all = np.sum((x - mean_all) ** 2)

    best_llr = -np.inf
    best_t = None
    best_subset = None

    for t in range(1, n):
        seg1 = x[:t]
        seg2 = x[t:]
        mean1 = np.mean(seg1)
        mean2 = np.mean(seg2)
        sse1 = np.sum((seg1 - mean1) ** 2)
        sse2 = np.sum((seg2 - mean2) ** 2)
        sse_split = sse1 + sse2
        llr = sse_all - sse_split
        if llr > best_llr:
            best_llr = llr
            best_t = t
            best_subset = x[:t]

    return best_llr, best_t, best_subset

In [48]:
results = {}

# Process the data on a per-country basis using 'country_code'
for country, group in df.groupby('country_code'):
    group_sorted = group.sort_values('year')
    emissions = group_sorted['value'].values
    if len(emissions) < 2:
        continue

    observed_llr, best_index, anomalous_subset = compute_llr(emissions)
    
    # Extract all years corresponding to the anomalous subset
    anomalous_years = group_sorted['year'].iloc[:best_index].tolist()
    best_year = group_sorted.iloc[best_index]['year']

    num_permutations = 1000
    llr_permutations = np.zeros(num_permutations)
    for i in range(num_permutations):
        emissions_perm = np.random.permutation(emissions)
        llr_perm, _, _ = compute_llr(emissions_perm)
        llr_permutations[i] = llr_perm if llr_perm is not None else -np.inf

    p_value = np.mean(llr_permutations >= observed_llr)

    results[country] = {
        'observed_llr': observed_llr,
        'best_change_point_index': best_index,
        'best_year': best_year,
        'anomalous_years': anomalous_years,
        'p_value': p_value,
        'n_points': len(emissions)
    }

# Filter and sort results: only include countries with a p-value < 0.05, sorted by LLR descending.
filtered_sorted_results = sorted(
    [(country, res) for country, res in results.items() if res['p_value'] < 0.05],
    key=lambda item: item[1]['observed_llr'], reverse=True
)

# Display the filtered and sorted results per country
for country, res in filtered_sorted_results:
    print(f"\nCountry: {country}")
    print(f"  Observed LLR: {res['observed_llr']}")
    print(f"  Best Change Point Index: {res['best_change_point_index']} (Year: {res['best_year']})")
    print(f"  Anomalous Years: {res['anomalous_years']}")
    print(f"  p-value from permutation testing: {res['p_value']}")
    print(f"  Number of Data Points: {res['n_points']}")
